In [1]:
import duckdb as ddb
import pandas as pd

con = ddb.connect("../air_quality.db")

df = con.query("select * from raw.air_quality where parameter in ('so2', 'pm10', 'pm25')").to_df()
df.head()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59615 entries, 0 to 59614
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   location_id         59615 non-null  int64         
 1   sensors_id          59615 non-null  int64         
 2   location            59615 non-null  object        
 3   datetime            59615 non-null  datetime64[us]
 4   lat                 59615 non-null  float64       
 5   lon                 59615 non-null  float64       
 6   parameter           59615 non-null  object        
 7   units               59615 non-null  object        
 8   value               59615 non-null  float64       
 9   month               59615 non-null  object        
 10  year                59615 non-null  int64         
 11  ingestion_datetime  59615 non-null  datetime64[us]
dtypes: datetime64[us](2), float64(3), int64(3), object(4)
memory usage: 5.5+ MB


In [5]:
df.describe()

,location_id,sensors_id,datetime,lat,lon,value,year,ingestion_datetime
count,5.961500e+04,5.961500e+04,59615,59615.000000,59615.000000,59615.000000,59615.000000,59615
mean,2.369309e+05,1.355969e+06,2024-01-31 18:16:08.252956,40.649504,-111.911877,7.715944,2023.754089,2024-12-18 11:36:34.516624
min,2.880000e+02,5.590000e+02,2023-12-01 08:00:00,40.341400,-112.355000,-4.000000,2023.000000,2024-12-18 11:35:40.514000
25%,2.168000e+03,3.897000e+03,2024-01-01 18:00:00,40.496408,-111.945000,0.200000,2024.000000,2024-12-18 11:36:01.238000
50%,8.758000e+03,2.553600e+04,2024-01-30 18:00:00,40.733501,-111.894167,3.200000,2024.000000,2024-12-18 11:36:35.767000
75%,2.283040e+05,1.316195e+06,2024-03-02 10:00:00,40.777100,-111.871696,9.900000,2024.000000,2024-12-18 11:37:02.248000
max,2.027500e+06,7.978757e+06,2024-04-01 06:00:00,40.897999,-111.711260,1343.000000,2024.000000,2024-12-18 11:37:54.713000
std,5.217907e+05,2.350080e+06,NaN,0.162539,0.141319,15.501408,0.430630,NaN


In [6]:
df.describe(include='O')

,location,parameter,units,month
count,59615,59615,59615,59615
unique,18,3,2,4
top,Utah Tech. Center-240600,pm25,µg/m³,01
freq,8346,36787,48290,15786


In [7]:
df[df.duplicated(subset=["location_id", "parameter", "units", "value", "datetime"])]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
54656,1236049,7970867,900S Sunnyside-2490680,2024-03-09 18:00:00,40.749303,-111.849583,pm10,µg/m³,7.150974,03,2024,2024-12-18 11:37:32.904
54657,1236049,7970867,900S Sunnyside-2490680,2024-03-09 19:00:00,40.749303,-111.849583,pm10,µg/m³,6.747727,03,2024,2024-12-18 11:37:32.904
57310,2026421,7978101,Constellation Labs Pleasant Grove-1996396,2024-03-09 18:00:00,40.357937,-111.711260,pm10,µg/m³,8.706845,03,2024,2024-12-18 11:37:48.960
57311,2026421,7978101,Constellation Labs Pleasant Grove-1996396,2024-03-09 19:00:00,40.357937,-111.711260,pm10,µg/m³,6.348214,03,2024,2024-12-18 11:37:48.960


In [8]:
df.groupby(by="parameter", as_index=False).count()

,parameter,location_id,sensors_id,location,datetime,lat,lon,units,value,month,year,ingestion_datetime
0,pm10,11503,11503,11503,11503,11503,11503,11503,11503,11503,11503,11503
1,pm25,36787,36787,36787,36787,36787,36787,36787,36787,36787,36787,36787
2,so2,11325,11325,11325,11325,11325,11325,11325,11325,11325,11325,11325


In [9]:
df[df["value"] < 0]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
271,288,559,Hawthorne-288,2023-12-01 22:00:00,40.733501,-111.871696,pm10,µg/m³,-4.0,12,2023,2024-12-18 11:35:40.514
303,288,559,Hawthorne-288,2023-12-08 13:00:00,40.733501,-111.871696,pm10,µg/m³,-1.0,12,2023,2024-12-18 11:35:40.514
344,288,559,Hawthorne-288,2023-12-03 11:00:00,40.733501,-111.871696,pm10,µg/m³,-1.0,12,2023,2024-12-18 11:35:40.514
361,288,559,Hawthorne-288,2023-12-04 04:00:00,40.733501,-111.871696,pm10,µg/m³,-2.0,12,2023,2024-12-18 11:35:40.514
388,288,559,Hawthorne-288,2023-12-04 09:00:00,40.733501,-111.871696,pm10,µg/m³,-2.0,12,2023,2024-12-18 11:35:40.514
...,...,...,...,...,...,...,...,...,...,...,...,...
50010,228304,1562372,Utah Tech. Center-240600,2024-03-31 07:00:00,40.777100,-111.945000,pm10,µg/m³,-1.0,03,2024,2024-12-18 11:37:06.513
50015,228304,1562372,Utah Tech. Center-240600,2024-03-31 12:00:00,40.777100,-111.945000,pm10,µg/m³,-4.0,03,2024,2024-12-18 11:37:06.513
50018,228304,1562372,Utah Tech. Center-240600,2024-03-31 15:00:00,40.777100,-111.945000,pm10,µg/m³,-3.0,03,2024,2024-12-18 11:37:06.513
50028,228304,1562372,Utah Tech. Center-240600,2024-04-01 01:00:00,40.777100,-111.945000,pm10,µg/m³,-2.0,03,2024,2024-12-18 11:37:06.513


In [10]:
con.close()